In [ ]:
pip install plotly

In [1]:
# Import libraries necessary for this project
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import libraries to data exploration
from sklearn.feature_selection import VarianceThreshold
import seaborn as sns

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py 
import plotly.graph_objs as go
import scipy.cluster.hierarchy as shc
from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation
from sklearn.preprocessing import StandardScaler
import os
import pickle as pk
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import sys
sys.path.insert(0, '/Users/maceli/useful_functions')

from functions import *

In [2]:
# Load the customers dataset
df=pd.read_sas("a2z_insurance.sas7bdat")

In [3]:
# Display the 10th rows of the dataset
display(df.head())

,CustID,FirstPolYear,BirthYear,EducDeg,MonthSal,GeoLivArea,Children,CustMonVal,ClaimsRate,PremMotor,PremHousehold,PremHealth,PremLife,PremWork
0,1.0,1985.0,1982.0,b'2 - High School',2177.0,1.0,1.0,380.97,0.39,375.85,79.45,146.36,47.01,16.89
1,2.0,1981.0,1995.0,b'2 - High School',677.0,4.0,1.0,-131.13,1.12,77.46,416.20,116.69,194.48,106.13
2,3.0,1991.0,1970.0,b'1 - Basic',2277.0,3.0,0.0,504.67,0.28,206.15,224.50,124.58,86.35,99.02
3,4.0,1990.0,1981.0,b'3 - BSc/MSc',1099.0,4.0,1.0,-16.99,0.99,182.48,43.35,311.17,35.34,28.34
4,5.0,1986.0,1973.0,b'3 - BSc/MSc',1763.0,4.0,1.0,35.23,0.90,338.62,47.80,182.59,18.78,41.45


In [4]:
# Display the type of features
df.dtypes

CustID           float64
FirstPolYear     float64
BirthYear        float64
EducDeg           object
MonthSal         float64
GeoLivArea       float64
Children         float64
CustMonVal       float64
ClaimsRate       float64
PremMotor        float64
PremHousehold    float64
PremHealth       float64
PremLife         float64
PremWork         float64
dtype: object

In [5]:
#see missing in the variables
summary_statistics(df)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
CustID,10296.0,NaN,NaN,NaN,5148.5,2972.34352,1.0,2574.75,5148.5,7722.25,10296.0
FirstPolYear,10266.0,NaN,NaN,NaN,1991.062634,511.267913,1974.0,1980.0,1986.0,1992.0,53784.0
BirthYear,10279.0,NaN,NaN,NaN,1968.007783,19.709476,1028.0,1953.0,1968.0,1983.0,2001.0
EducDeg,10279,4,b'3 - BSc/MSc',4799,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MonthSal,10260.0,NaN,NaN,NaN,2506.667057,1157.449634,333.0,1706.0,2501.5,3290.25,55215.0
GeoLivArea,10295.0,NaN,NaN,NaN,2.709859,1.266291,1.0,1.0,3.0,4.0,4.0
Children,10275.0,NaN,NaN,NaN,0.706764,0.455268,0.0,0.0,1.0,1.0,1.0
CustMonVal,10296.0,NaN,NaN,NaN,177.892605,1945.811505,-165680.42,-9.44,186.87,399.7775,11875.89
ClaimsRate,10296.0,NaN,NaN,NaN,0.742772,2.916964,0.0,0.39,0.72,0.98,256.2
PremMotor,10262.0,NaN,NaN,NaN,300.470252,211.914997,-4.11,190.59,298.61,408.3,11604.42


In [6]:
missing_data(df)

,Total,Percent
PremLife,104,1.010101
PremWork,86,0.835276
PremHealth,43,0.417638
MonthSal,36,0.349650
PremMotor,34,0.330225
FirstPolYear,30,0.291375
Children,21,0.203963
BirthYear,17,0.165113
EducDeg,17,0.165113
GeoLivArea,1,0.009713


In [13]:
#Display the shape of dataset #features and #obs
df.shape

(10296, 14)

In [15]:
# Check duplicated Customer Identify
df['CustID'].duplicated().sum()

0

In [17]:
#Checking the data integrity
df=df[df['BirthYear']<df['FirstPolYear']]
df.shape

(8092, 14)

In [ ]:
df['Has Children (Y=1)']=df['Has Children (Y=1)'].replace(1.,'1')
df['Has Children (Y=1)']=df['Has Children (Y=1)'].replace(0.,'0')
df['Has Children (Y=1)']=df['Has Children (Y=1)'].replace(np.nan,'Sem Informacao')
df['Has Children (Y=1)'].unique()
df['Geographic Living Area']=df['Geographic Living Area'].replace(np.nan,'Not Information')
df['Geographic Living Area']=df['Geographic Living Area'].replace(1.0,'1')
df['Geographic Living Area']=df['Geographic Living Area'].replace(2.0,'2')
df['Geographic Living Area']=df['Geographic Living Area'].replace(3.0,'3')
df['Geographic Living Area']=df['Geographic Living Area'].replace(4.0,'4')
df['Educational Degree']=df['Educational Degree'].replace(np.nan,'Not Information')
df['Educational Degree'].unique()
df['Geographic Living Area'].unique()

In [34]:
c = ['GeoLivArea','Children','Children']
df[c] = df[c].astype(str)

In [32]:
df.dtypes

CustID           float64
FirstPolYear     float64
BirthYear        float64
EducDeg           object
MonthSal         float64
GeoLivArea        object
Children         float64
CustMonVal       float64
ClaimsRate       float64
PremMotor        float64
PremHousehold    float64
PremHealth       float64
PremLife         float64
PremWork         float64
dtype: object

In [26]:
c = ['PremHealth','PremHousehold','PremLife','PremMotor','PremWork','ClaimsRate','CustMonVal', 'MonthSal']
df[c] = df[c].fillna(df[c].mean())

In [ ]:
#The current year of the database is 2016
df['current year']=2016
df['Age']=df['current year']-df['Brithday Year'] 
df['Age Policy']=df['First Policy´s Year']-df['Brithday Year'] 
df['Policy year']=df['current year']-df['First Policy´s Year'] 
df.head()

In [ ]:
df.dtypes
df_without=df.copy()
df_without.head()

In [ ]:
#Outliers
def drop_outliers(df, field_name):
    distance = 1.5 * (np.nanpercentile(df[field_name], 75) - np.nanpercentile(df[field_name], 25))
    df.drop(df[df[field_name] > distance + np.nanpercentile(df[field_name], 75)].index, inplace=True)
    df.drop(df[df[field_name] < np.nanpercentile(df[field_name], 25) - distance].index, inplace=True)

if __name__ == "__main__":
  # assuming df exists and contains numeric variables

  print(df.shape)

for column in df.select_dtypes(include=[np.number]).columns:
      drop_outliers(df, column)

print(df.shape)

In [ ]:
f,axes = plt.subplots(2, 2, figsize=(8, 8))
sns.despine(left=True)
sns.boxplot(y='Premiums in LOB: Work Compensations', x='Geographic Living Area',
                 data=df_without, ax=axes[0, 0])
sns.boxplot(y='Premiums in LOB: Work Compensations', x='Geographic Living Area',
                 data=df, ax=axes[0, 1])
sns.boxplot(y='Premiums in LOB:  Life', x='Geographic Living Area',
                 data=df_without, ax=axes[1, 0])
sns.boxplot(y='Premiums in LOB:  Life', x='Geographic Living Area',
                 data=df, ax=axes[1, 1])
plt.savefig('medals.svg')
#sns.boxplot(y='Premiums in LOB: Health', x='Geographic Living Area',
                 #data=df, ax=axes[0, 2])
#sns.boxplot(y='Premiums in LOB: Household', x='Geographic Living Area',
                 #data=df, ax=axes[1, 0])
#sns.boxplot(y='Claims Rate', x='Geographic Living Area',
                 #data=df, ax=axes[1, 1])
#sns.boxplot(y='Customer Monetary Value', x='Geographic Living Area',
                 #data=df, ax=axes[1, 2])
plt.tight_layout()

In [ ]:

#Outliers
def drop_outliers(df, field_name):
    distance = 1.5 * (np.nanpercentile(df[field_name], 75) - np.nanpercentile(df[field_name], 25))
    df.drop(df[df[field_name] > distance + np.nanpercentile(df[field_name], 75)].index, inplace=True)
    df.drop(df[df[field_name] < np.nanpercentile(df[field_name], 25) - distance].index, inplace=True)

if __name__ == "__main__":
  # assuming df exists and contains numeric variables

  print(df.shape)

for column in df.select_dtypes(include=[np.number]).columns:
      drop_outliers(df, column)

print(df.shape)

In [ ]:
f,axes = plt.subplots(2, 3, figsize=(8, 8))
sns.despine(left=True)
sns.boxplot(y='Premiums in LOB: Work Compensations', x='Geographic Living Area',
                 data=df, ax=axes[0, 0])
sns.boxplot(y='Premiums in LOB:  Life', x='Geographic Living Area',
                 data=df, ax=axes[0, 1])
sns.boxplot(y='Premiums in LOB: Health', x='Geographic Living Area',
                 data=df, ax=axes[0, 2])
sns.boxplot(y='Premiums in LOB: Household', x='Geographic Living Area',
                 data=df, ax=axes[1, 0])
sns.boxplot(y='Claims Rate', x='Geographic Living Area',
                 data=df, ax=axes[1, 1])
sns.boxplot(y='Customer Monetary Value', x='Geographic Living Area',
                 data=df, ax=axes[1, 2])
plt.tight_layout()

In [ ]:
f,axes = plt.subplots(2, 3, figsize=(8, 8))
sns.despine(left=True)
sns.boxplot(y='Premiums in LOB: Work Compensations', x='Educational Degree',
                 data=df, ax=axes[0, 0])
sns.boxplot(y='Premiums in LOB:  Life', x='Educational Degree',
                 data=df, ax=axes[0, 1])
sns.boxplot(y='Premiums in LOB: Health', x='Educational Degree',
                 data=df, ax=axes[0, 2])
sns.boxplot(y='Premiums in LOB: Household', x='Educational Degree',
                 data=df, ax=axes[1, 0])
sns.boxplot(y='Claims Rate', x='Educational Degree',
                 data=df, ax=axes[1, 1])
sns.boxplot(y='Customer Monetary Value', x='Educational Degree',
                 data=df, ax=axes[1, 2])
plt.tight_layout()

In [ ]:
keep=['Premiums in LOB: Work Compensations','Premiums in LOB:  Life','Premiums in LOB: Health',
      'Premiums in LOB: Household','Premiums in LOB: Motor','Claims Rate','Customer Monetary Value',
      'Gross Monthly Salary','Age','Age Policy','Policy year'
     ]
X_cor=df[keep]

In [ ]:
constant_filter = VarianceThreshold(threshold=0)
constant_filter.fit(X_cor)
len(X_cor.columns[constant_filter.get_support()])
constant_columns = [column for column in X_cor.columns
                    if column not in X_cor.columns[constant_filter.get_support()]]

print(len(constant_columns))
for column in constant_columns:
    print(column)
#Neste caso todas as variaveis permanecem

In [ ]:
correlated_features = set()
correlation_matrix =X_cor.corr()
for i in range(len(correlation_matrix .columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)
len(correlated_features)
print(correlated_features)



In [ ]:
mask = np.zeros_like(X_cor.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
plt.subplots(figsize = (15,15))
sns.heatmap(X_cor.corr(), 
            annot=True,
            mask = mask,
            cmap = plt.cm.Reds,
            linewidths=0.1, 
            linecolor='white',
            vmax = .9,
            square=True)
plt.title("Correlations Among Features", y = 1.03,fontsize = 5)
plt.show()
plt.savefig('medals.svg')

In [ ]:
import pickle as pk
df_end=df.drop(['Age Policy', 'Premiums in LOB: Motor', 'Age', 'Customer Monetary Value'],axis=1)

In [ ]:
df_end.describe()

In [ ]:
df_end.to_pickle('df_end.pkl')

In [ ]:
df_dsenv=pd.read_pickle('df_end.pkl')

In [ ]:
df_dsenv.groupby(['Geographic Living Area'])['Gross Monthly Salary','Premiums in LOB: Household',
    'Premiums in LOB: Health','Premiums in LOB: Work Compensations','Claims Rate',
                                         'Premiums in LOB:  Life'].mean()

In [ ]:
df_dsenv.groupby(['Educational Degree'])['Gross Monthly Salary','Premiums in LOB: Household',
    'Premiums in LOB: Health','Premiums in LOB: Work Compensations','Claims Rate',
                                         'Premiums in LOB:  Life'].mean()

In [ ]:
keep=['Premiums in LOB: Work Compensations','Premiums in LOB:  Life','Premiums in LOB: Health',
      'Premiums in LOB: Household']
X = StandardScaler().fit_transform(df_dsenv[keep])

In [ ]:
# Fit KMeans and calculate SSE for each *k*
sse = {}
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=1)
    kmeans.fit(X)
    sse[k] = kmeans.inertia_
# Plot SSE for each *k*
plt.title('The Elbow Method')
plt.xlabel('k'); plt.ylabel('SSE')
sns.pointplot(x=list(sse.keys()), y=list(sse.values()))
plt.show()

plt.savefig('plot.png')
#Ideally, we want to choose the point on the elbow chart where the
#SSE stops decreasing at an increasing rate — i.e. the point where the change gradient of between the number 
#of clusters becomes constant. For our model we will choose k=2-3.

In [ ]:
kmeans_2 = KMeans(n_clusters = 2, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
y_kmeans_2 = kmeans_2.fit_predict(X)
kmeans_3 = KMeans(n_clusters = 3, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
y_kmeans_3 = kmeans_3.fit_predict(X)

# Visualising the clusters
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set(context="notebook", palette="Spectral", style = 'darkgrid' ,font_scale = 1.5, color_codes=True)

sns.scatterplot(X[y_kmeans_2 == 0, 0], X[y_kmeans_2 == 0, 1], color = 'yellow', label = 'Cluster 1',s=50)
sns.scatterplot(X[y_kmeans_2 == 1, 0], X[y_kmeans_2 == 1, 1], color = 'blue', label = 'Cluster 2',s=50)
sns.scatterplot(kmeans_2.cluster_centers_[:, 0], kmeans_2.cluster_centers_[:, 1], color = 'red', 
               label = 'Centroids',s=300,marker=',')

#We need do a subplot 
#sns.scatterplot(X[y_kmeans_3 == 0, 0], X[y_kmeans_3 == 0, 1], color = 'yellow', label = 'Cluster 1',s=50)
#sns.scatterplot(X[y_kmeans_3 == 1, 0], X[y_kmeans_3 == 1, 1], color = 'blue', label = 'Cluster 2',s=50)
#sns.scatterplot(X[y_kmeans_3 == 3, 0], X[y_kmeans_3 == 3, 1], color = 'green', label = 'Cluster 3',s=50)
#sns.scatterplot(kmeans_3.cluster_centers_[:, 0], kmeans_3.cluster_centers_[:, 1], color = 'red', 
              # label = 'Centroids',s=300,marker=',')

In [ ]:
df_dsenv['Cluster_Id'] = kmeans_2.labels_
centers_2 = kmeans_2.cluster_centers_
centers_2

In [ ]:
def pd_centers(featuresUsed, centers):
    colNames = list(featuresUsed)
    colNames.append('prediction')

    # Zip with a column called 'prediction' (index)
    Z = [np.append(A, index) for index, A in enumerate(centers)]

    # Convert to pandas data frame for plotting
    P = pd.DataFrame(Z, columns=colNames)
    P['prediction'] = P['prediction'].astype(int)
    return P

P = pd_centers(keep, centers_2)

# Function that creates Parallel Plots
from itertools import cycle, islice
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
def parallel_plot(data):
    my_colors = list(islice(cycle(['b', 'r', 'g', 'y', 'k']), None, len(data)))
    plt.figure(figsize=(15,8)).gca().axes.set_ylim([-3,+3])
    parallel_coordinates(data, 'prediction', color = my_colors, marker='o')
    
parallel_plot(P)

In [ ]:

cluster_avg=df_dsenv.groupby(['Cluster_Id'])['Premiums in LOB: Household',
    'Premiums in LOB: Health','Premiums in LOB: Work Compensations',
                                         'Premiums in LOB:  Life'].mean()
pop_avg=df_dsenv[keep].mean()

relative_imp = cluster_avg / pop_avg - 1

# Initialize a plot with a figure size of 8 by 2 inches 
plt.figure(figsize=(4, 2))
# Add the plot title
plt.title('Relative importance of attributes')
# Plot the heatmap
sns.heatmap(data=relative_imp, annot=True, fmt='.2f', cmap='RdYlGn')
plt.show()

In [ ]:
cluster_avg=pd.DataFrame(df_dsenv.groupby(['Cluster_Id'])['Premiums in LOB: Household',
    'Premiums in LOB: Health','Premiums in LOB: Work Compensations',
                                         'Premiums in LOB:  Life'].mean())

cluster_avg
total_avg = pd.DataFrame(df_dsenv[keep].mean(), columns=['Pop Avg']).T
total_avg
cluster_mean =total_avg.append([cluster_avg])
cluster_mean
values = cluster_mean.iloc[0].tolist() #
values += values [:1]

angles = [n / float(AttNo) * 2 * pi for n in range(AttNo)]
angles += angles [:1]

values2 = cluster_mean.iloc[1].tolist() # 
values2 += values2 [:1]

angles2 = [n / float(AttNo) * 2 * pi for n in range(AttNo)]
angles2 += angles2 [:1]

values3 = cluster_mean.iloc[2].tolist() # 
values3 += values3 [:1]

angles3 = [n / float(AttNo) * 2 * pi for n in range(AttNo)]
angles3 += angles3 [:1]

plt.figure(figsize=(5,5))
ax = plt.subplot(111, polar=True)

plt.xticks(angles[:-1],Attributes)
# Group1
ax.plot(angles, values, color = 'r')
ax.fill(angles, values, 'red', alpha=0.1)

# Group2
ax.plot(angles2, values2, color = 'b')
ax.fill(angles2, values2, 'blue', alpha=0.1)


# Total Average
ax.plot(angles3, values3, color = 'black')
ax.fill(angles3, values3, 'black', alpha=0.1)

plt.figtext(0.5,0.63,"Avg Pop",color="red")
plt.figtext(0.4,0.4,"Premium = 0",color="blue")
plt.figtext(0.4,0.8,"Premium = 1",color="green")
plt.show()

In [ ]:
df_dsenv.head()

In [ ]:
df_dsenv.groupby(['Geographic Living Area','Educational Degree','Cluster_Id'])['Cluster_Id'].count()

In [ ]:
keep=['Gross Monthly Salary','Claims Rate','Cluster_Id']
dummies=['Educational Degree','Geographic Living Area','Has Children (Y=1)']
dummies=pd.get_dummies(df_dsenv[dummies], drop_first=True)
dummies.shape
dummies.head()
df_dsenv=pd.concat([df_dsenv[keep], dummies],axis=1)

In [ ]:
df_dsenv.head()

In [ ]:

y=df_dsenv['Cluster_Id']
y.head()
y.shape
X=df_dsenv.drop(['Cluster_Id'],axis=1)
X.head()

In [ ]:
#Divindo a amostra em validacao e treino, neste caso usou-se a proporcao 70/30

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
clf = DecisionTreeClassifier(max_depth=3)
clf.fit(X_train,y_train)
y_pred = clf.predict( X_test)

In [ ]:
pred_scores = clf.predict_proba(X_test)[:,1]
X_test['pred']=pred_scores
X_test.head()

In [ ]:
importances = pd.DataFrame({'feature':X.columns,'importance':np.round(clf.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False)
importances 

In [ ]:
keep_2=['Claims Rate','Gross Monthly Salary','Has Children (Y=1)_1','pred']
df_vali=pd.concat([X_test[keep_2],y_test],axis=1)
df_vali

In [ ]:
df_vali.isnull().sum()

In [ ]:
np.percentile(df_vali['pred'], [10,25,55,80,90], axis=0)

In [ ]:
df_vali.groupby(['Has Children (Y=1)_1'])['pred'].mean()

In [ ]:
df_vali['pred'].describe()

In [ ]:
def f(row):
    if row['pred'] <.17:
        val = 0
    elif row['pred'] >= .17 and row['pred'] <.28:
        val = 1
    elif row['pred'] >= .28 and row['pred'] <.36:
        val = 2
    elif row['pred'] >= .36 and row['pred'] <.59:
        val = 3
    elif row['pred'] >= .59 and row['pred'] <.89:
        val = 4
    else:
        val = 5
    return val
df_vali['as'] = df_vali.apply(f, axis=1)
df_vali.head()

In [ ]:
df_vali.groupby(['as'])['Cluster_Id'].count()
#df_vali.groupby(['Has Children (Y=1)_1','as'])['Claims Rate'].mean()
df_vali.groupby(['as'])['Claims Rate'].mean()
#df_vali.groupby(['as'])['Gross Monthly Salary'].mean()
#df_vali.groupby(['Has Children (Y=1)_1','as']).count()
#df_vali.groupby(['Has Children (Y=1)_1','as'])["Gross Monthly Salary"].mean()

In [ ]:
y_new=df_dsenv['Cluster_Id']
y_new.head()
y_new.shape
X_new=df_dsenv.drop(['Cluster_Id',"Claims Rate"],axis=1)
X_new.head()
keep_1=['Age']
X_new=pd.concat([X_new,df[keep_1]],axis=1)

In [ ]:
X_new.head()

In [ ]:
import numpy
from sklearn import linear_model
cls = RandomForestRegressor()#c

from sklearn.model_selection import train_test_split
X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(X_new, y_new, test_size=0.2, random_state=123)
cls.fit(X_train_n, y_train_n)
scoreOfModel = cls.score(X_train_n, y_train_n)
